# Grounding Guardrail 

### Fact-Checking

The goal of the self-check fact-checking output rail is to ensure that the answer to a RAG (Retrieval Augmented Generation) query is grounded in the provided evidence extracted from the knowledge base (KB).

NeMo Guardrails uses the concept of **relevant chunks** (which are stored in the `$relevant_chunks` context variable) as the evidence against which fact-checking should be performed. The relevant chunks can be extracted automatically, if the built-in knowledge base support is used, or provided directly alongside the query (see the [Getting Started Guide example](../getting_started/7_rag)).

**IMPORTANT**: The performance of this rail is strongly dependent on the capability of the LLM to follow the instructions in the `self_check_facts` prompt.

In [25]:
import sys
import os

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [26]:
import json
import boto3
from utils import bedrock, print_ww
import pandas as pd

# This helper function encompasses the process of initializing NeMo Guardrails and generating Rails based on a specified configuration.
from nemoguardrails import LLMRails, RailsConfig

# BedrockModels is a "Singleton" class which initializes the necessary models for the notebook.
from models import BedrockModels

# This creates a pywidget chat window for testing
from models import ChatComponent

In [27]:
! pip show nemoguardrails

Name: nemoguardrails
Version: 0.7.1
Summary: NeMo Guardrails is an open-source toolkit for easily adding programmagle guardrails to LLM-based conversational systems.
Home-page: 
Author: 
Author-email: NVIDIA <nemoguardrails@nvidia.com>
License: SPDX-FileCopyrightText: Copyright (c) 2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
        SPDX-License-Identifier: Apache-2.0
        
        Licensed under the Apache License, Version 2.0 (the "License");
        you may not use this file except in compliance with the License.
        You may obtain a copy of the License at
        
        http://www.apache.org/licenses/LICENSE-2.0
        
        Unless required by applicable law or agreed to in writing, software
        distributed under the License is distributed on an "AS IS" BASIS,
        WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
        See the License for the specific language governing permissions and
        limitations under the Licens

## Initalize bedrock client

In [28]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Define function to create a LLMRails instance with Claudev2 as LLM and a certain rail configuration

In [29]:
# This helper function encapsulates the necessary steps to bootstrap
# NeMo Guardrails and returns Rails based on a given configuration.
def bootstrap_bedrock_nemo_guardrails(rail_config_path: str) -> LLMRails:

    #1. initialize rails config
    config = RailsConfig.from_path(f"NeMo/rails/{rail_config_path}/config")

    # initialize bedrock models
    # you can pass model id as string or use the default model id 'anthropic.claude-v2'
    bedrock_models = BedrockModels
    bedrock_models.init_bedrock_client(boto3_bedrock)
    bedrock_models.init_llm('anthropic.claude-v2')

    # 2. bootstraps NeMo Guardrails with the necessary resources
    app = LLMRails(config=config, llm=bedrock_models.llm,  verbose=True)
    
    return app


In [30]:
rails = bootstrap_bedrock_nemo_guardrails("grounding")

Entered verbose mode.


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
import nest_asyncio
nest_asyncio.apply()

In [32]:
response = await rails.generate_async(messages=[
    {"role": "context", "content": {"check_facts": False}},
    {
    "role": "user",
    "content": "How many vacation days do I have? as a regular employee"}])

print_ww(response["content"])

Event UtteranceUserActionFinished {'final_transcript': 'How many vacation days do I have? as a regular employee'}
Event StartInternalSystemAction {'uid': '50d72db4-ac2a-408c-8c93-24d243cb2679', 'event_created_at': '2024-02-16T22:31:02.190377+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': '256b1f98-9e0f-4b8d-90ab-93e5c41568f2', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': 'b258609a-1c1a-4e3a-a0e1-7b8e7d0944e0', 'event_created_at': '2024-02-16T22:31:02.190697+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'How many vacation days do I have? as a regular employee'}
Event StartInternalSystemAction {'uid': 'b678e00b-7da8-489f-a31b-4f101e502b84', 'event_created_at': '2024-02-16T22:31:02.191133+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': N

In [12]:
for call in range(len(rails.explain().llm_calls)):
    call_metadata = rails.explain().llm_calls[call]
    
    print_ww(f"task: {call+1}", call_metadata.task)
    print_ww(f"prompt: {call+1}", call_metadata.prompt)
    print_ww(f"completion: {call+1}", call_metadata.completion)

    print("------\n")

task: 1 generate_user_intent
prompt: 1 System: """
Below is a conversation between a user and a bot called the ABC Bot.
The bot is designed to answer employee questions about the ABC Company.
The bot is knowledgeable about the employee handbook and company policies.
If the bot does not know the answer to a question, it truthfully says it does not know.

Your task is to generate a short summary called user intent for the last user message in a
conversation.
"""

# This is how a conversation between a user and the bot can go:
human "Hi there. Can you help me with some questions I have about the company?"
express greeting and ask for assistance
assistant express greeting and confirm and offer assistance
"Hi there! I'm here to help answer any questions you may have about the ABC Company. What would you
like to know?"
human "What are the companies employee benefits?"
ask question about benefits
assistant respond to question about benefits
"The ABC Company provides comprehensive employee ben

In [35]:
rails.explain()

ExplainInfo(llm_calls=[LLMCallInfo(task='generate_user_intent', prompt='System: """\nBelow is a conversation between a user and a bot called the ABC Bot.\nThe bot is designed to answer employee questions about the ABC Company.\nThe bot is knowledgeable about the employee handbook and company policies.\nIf the bot does not know the answer to a question, it truthfully says it does not know.\n\nYour task is to generate a short summary called user intent for the last user message in a conversation.\n"""\n\n# This is how a conversation between a user and the bot can go:\nhuman "Hi there. Can you help me with some questions I have about the company?"\nexpress greeting and ask for assistance\nassistant express greeting and confirm and offer assistance\n"Hi there! I\'m here to help answer any questions you may have about the ABC Company. What would you like to know?"\nhuman "What\'s the company policy on paid time off?"\nask question about benefits\nassistant respond to question about benefits

In [65]:
response = rails.generate(messages=[{
    "role": "context",
    "content": {
        "relevant_chunks": """
            Employees are eligible for the following time off:
              * Vacation: 20 days per year, accrued monthly.
              * Sick leave: 15 days per year, accrued monthly.
              * Personal days: 5 days per year, accrued monthly.
              * Paid holidays: New Year's Day, Memorial Day, Independence Day, Thanksgiving Day, Christmas Day.
              * Bereavement leave: 3 days paid leave for immediate family members, 1 day for non-immediate family members. """
    }
},{
    "role": "user",
    "content": "What is the policy for office romance?"
}])
print(response["content"])

Event UtteranceUserActionFinished {'final_transcript': 'What is the policy for office romance?'}
Event StartInternalSystemAction {'uid': '88c139c5-242d-4ceb-af51-33df816354f9', 'event_created_at': '2024-02-13T15:46:16.939306+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': 'c84a700f-4d37-41a9-9dcd-6a44e3aa81e4', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': '7d877c3b-dca3-4f60-b289-487d7e53f57a', 'event_created_at': '2024-02-13T15:46:16.939607+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'What is the policy for office romance?'}
Event StartInternalSystemAction {'uid': '1724de5d-70a0-463e-a877-7b2a6187c836', 'event_created_at': '2024-02-13T15:46:16.940314+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '459353ca-0b5b-

## Test prompt without Grounding Guardrail which creates hallucination

In [ ]:
example_prompt = "Is it possible for cats to learn the names of their friends cats if they live in cat cafes?"

In [20]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


In [21]:
response = textgen_llm(example_prompt)

print_ww(response)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 I don't see why not. Cats can learn to recognize other cats that they frequently interact with,
just like humans learn to recognize people they know. In a cat cafe environment where the same cats
live together, they would have lots of opportunity to get to know each other and could potentially
learn each other's names if the cafe staff uses the cats' names when interacting with them. Cats
have good memories when it comes to familiar faces and sounds, so it's plausible they could
associate certain meows or human words with specific cats they regularly encounter. However, we
can't know for sure what level of abstract thinking cats are capable of. But associating names with
feline friends seems well within the realm of possibility for smarter, socialized cats.


## Test prompt with Grounding Guardrail

In [8]:
# Bootstrap Guardrails with grounding configuration
grounding_llm = bootstrap_bedrock_nemo_guardrails('grounding')


Human conversations vector store index loaded from disk.

Assistant conversations vector store index loaded from disk.

NeMo Conversations Flows vector store index loaded from disk.

KnowledgeBase vector store index loaded from disk.


In [9]:
bot_message = grounding_llm.generate(prompt="Is it possible for cats to learn the names of their friends cats if they live in cat cafes?")
print(bot_message)

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


Unfortunately I do not have enough context to definitively say whether cats living in cat cafes can learn the names of their friends' cats. However, here are some thoughts on this interesting question:
- Cats are intelligent animals with good memory, so they are capable of learning and remembering things like names. However, their ability and motivation to do so likely depends a lot on the individual cat.
- Cats that regularly interact with the same group of other cats at a cafe could potentially learn to recognize those familiar felines. However, cats interact primarily through scent, body language and vocalizations, so learning human-given names may not be a priority.
- A cat cafe environment provides lots of stimulation, which could make it more difficult for cats to focus on learning names specifically. But regular exposure to the same group of cats may allow for some name recognition over time.
- Training techniques like reward association could potentially be used by cafe staff t

In [10]:
bot_message = grounding_llm.generate(prompt="In which year did the DARPA announce a multi-year investment?")
print(bot_message)

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


Based on the context provided, DARPA announced a multi-year investment of more than $2 billion in 2018 as part of its AI Next campaign:
"In 2018 the Defense Advanced Research Project Agency (DARPA) announced a multi-year investment of more than $2 billion in new and existing programs and called it the AI Next campaign."


In [12]:

#bot_message = grounding_llm.generate(prompt="What was the unemployment rate for teenagers?")
#print(bot_message)

bot_message = grounding_llm.generate(prompt="I will start a new position as an engineer at AWS in Zurich, how many days of vacation can I expect to get?")
print(bot_message)

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock


Unfortunately I do not have specific details on the vacation policy for an engineering role at AWS in Zurich. However, here are a few points that may help provide some context:
- AWS generally offers competitive benefits, including vacation time, to attract and retain top talent. The exact number of vacation days likely depends on level/seniority of the role.
- Standard vacation policies at large technology companies for engineering roles often start at around 20 days per year and increase with seniority. Some companies also add additional paid days off for things like sick time and holidays.
- Policies and norms around vacation do vary significantly between the US and Europe. In general, European countries tend to have more statutory minimum vacation days. For Switzerland specifically, the legal minimum is 4 weeks (20 days) of paid vacation per year.
- Large international companies like AWS sometimes aim to provide roughly comparable vacation time across regions, while accounting for 

In [13]:
# asynchronous invocation with history
history.append(bot_message)
history.append(
    {"role": "user", "content": "What was the unemployment rate for teenagers?"}
)
bot_message = await grounding_llm.generate_async(messages=history)
print(bot_message['content'])

NameError: name 'history' is not defined

In [17]:
## Experiment with the Chat Interface and Review Sample Questions Ahead
#Please feel free to interact with the chat below. Once completed, proceed to the next block to examine sample questions.

#### Interactive session using ipywidgets

#The following utility class allows us to interact with the AI Assistant in a more natural way. We write out the question in an input box, and get the Assistant's answer. We can then continue our conversation.

In [11]:
grounding_chat = ChatComponent(llm=grounding_llm)
grounding_chat.render()

Box(children=(Image(value=b'GIF89a\xc8\x00\xc8\x00\xf7\x00\x00;Ch\x83\x90\xb7\xcf\xdc\xe8\xda\xec\xf1\xf1\xf2\…

In [22]:
history = [{"role": "user", "content": hallucination_example_prompt}]
bot_message = await grounding_llm.generate_async(messages=history)
print(bot_message['content'])

Parameter temperature does not exist for Bedrock
Parameter temperature does not exist for Bedrock
Hallucination rail can only be used with OpenAI LLM engines.Current LLM engine is NoneType.


<response>
- This Purina Pro Plan LiveClear Dry Cat Food for Kittens Chicken & Rice Formula is suitable for your active male cat weighing 2.26kg as it is a dry food suitable for cats of all breeds and life stages.
- As a cat food specially formulated for allergen management, this food is a good choice for your cat as it features ingredients derived from single protein sources, in this case chicken, to help support cats with food sensitivities and allergies. This makes it a hypoallergenic option.
- The primary protein of chicken is digestible and a quality source of protein to support your active cat's muscle growth and development while staying within an ideal weight.
- The dry kibble texture is convenient for both dry food feeders as well as cats that enjoy a mix of textures, as it can be easily served on its own or combined with wet foods.
- The allergen-managed formula is nutritionally complete to meet all your cat's dietary needs for a healthy immune system as he remains active ind